In [1]:
import pandas as pd
import json
import urllib
import requests as rq
import base64
from datetime import datetime

In [2]:
import config
API_KEY = config.API_KEY
API_SECRET = config.API_SECRET

https://stackoverflow.com/questions/40023931/how-to-get-real-estate-data-with-idealista-api

In [3]:
def get_oauth_token(): 
    url = "https://api.idealista.com/oauth/token"    
    apikey= API_KEY #sent by idealist
    secret= API_SECRET #sent by idealista
    apikey_secret = apikey + ':' + secret
    auth = str(base64.b64encode(bytes(apikey_secret, 'utf-8')))[2:][:-1]
    headers = {'Authorization' : 'Basic ' + auth,'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}
    params = urllib.parse.urlencode({'grant_type':'client_credentials'}) #,'scope':'read'
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']
    return bearer_token

In [4]:
def search_api(token, URL):  
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)
    return result

In [5]:
# values: es, it, pt
country = 'es' 
# search language for summary. values: es, it, pt, en, ca
locale = 'es' 
language = 'es' 
# items per page: 50 as maximum allowed
max_items = '50'
# values: sale, rent
operation = 'sale'
# values: homes, offices, premises, garages, bedrooms
property_type = 'homes'
# allowed values by property type:
# garages: distance, price, street, photos, publicationDate, modificationDate (rent only), weigh, pricedown
# premises: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2 (rent only), weigh, pricedown
# offices: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2, weigh, pricedown
# homes: distance, price, street, photos, publicationDate, modificationDate, size, floor, rooms, ratioeurm2 (sólo alquiler), weigh, pricedown
# rooms: distance, price, street, photos, publicationDate, modificationDate, floor
order = 'priceDown' 
# geographic coordinates (WGS84) (latitude,longitude)
center = '43.3773,-8.39852' 
# distance to center, in metres (ratio)
distance = '5000'
# values: asc, desc
sort = 'desc'
# 
maxPrice = '300000'
# numPage = 1

In [6]:
df = pd.DataFrame()
for i in range(1,15):
    numPage = str(i)
    url = ('https://api.idealista.com/3.5/'+country+
            '/search?operation='+operation+
            '&locale='+locale+
            '&maxItems='+max_items+
            '&order='+order+
            '&center='+center+
            '&distance='+distance+
            '&maxPrice='+maxPrice+
            '&propertyType='+property_type+
            '&sort='+sort+ 
            '&numPage='+numPage+
            '&language='+language)
    result = search_api(get_oauth_token(), url)
    df_temp = pd.json_normalize(result, record_path = ["elementList"])
    df = pd.concat([df, df_temp])
now = datetime.now()
date_time = now.strftime("%m%d%Y_%H:%M")
df.to_csv('idealista_search' + str(date_time) + '.csv')

In [7]:
print(len(df))
df

700


,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,detailedType.typology,suggestedTexts.subtitle,suggestedTexts.title,neighborhood,parkingSpace.hasParkingSpace,parkingSpace.isParkingSpaceIncludedInPrice,detailedType.subTypology,parkingSpace.parkingSpacePrice,labels,newDevelopmentFinished
0,98508410,https://img3.idealista.com/blur/WEB_LISTING/0/...,22/063,10,5,75000.0,flat,sale,54.0,True,...,flat,"Monte Alto - Zalaeta - Atocha, A Coruña",Piso en Calle Valencia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,99744246,https://img3.idealista.com/blur/WEB_LISTING/0/...,P.A/Ronda de Nelle,15,3,250000.0,flat,sale,60.0,False,...,flat,"Paseo de los Puentes-Santa Margarita, A Coruña","Piso en Ronda nelle, 142",Paseo de los Puentes-Santa Margarita,NaN,NaN,NaN,NaN,NaN,NaN
2,97943866,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,24,3,115000.0,flat,sale,85.0,True,...,flat,"Agra del Orzán - Ventorrillo, A Coruña","Piso en Gil Vicente, 4",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99428153,https://img3.idealista.com/blur/WEB_LISTING/0/...,B250,10,2,98000.0,flat,sale,88.0,False,...,flat,"Os Mallos, A Coruña","Piso en Avenida dos Mallos, 98",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99019049,https://img3.idealista.com/blur/WEB_LISTING/0/...,K26,20,1,249000.0,flat,sale,90.0,True,...,flat,"Monte Alto - Zalaeta - Atocha, A Coruña",Piso en Calle Socorro s/n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,98886153,https://img3.idealista.com/blur/WEB_LISTING/0/...,3-F,20,3,285000.0,flat,sale,66.0,True,...,flat,"Monte Alto - Zalaeta - Atocha, A Coruña","Piso en Calle Adelaida Muro, 1",NaN,True,True,NaN,NaN,NaN,True
46,27531680,https://img3.idealista.com/blur/WEB_LISTING/0/...,1963,37,9,300000.0,penthouse,sale,185.0,False,...,flat,"Agra del Orzán - Ventorrillo, A Coruña",Ático en Calle Barcelona,NaN,NaN,NaN,penthouse,NaN,NaN,NaN
47,95038714,https://img3.idealista.com/blur/WEB_LISTING/0/...,Portal 5. 1º C,29,1,210000.0,flat,sale,96.0,True,...,flat,"Eirís, A Coruña","Piso en Calle Dabán, 1",NaN,True,True,NaN,NaN,NaN,True
48,97507233,https://img3.idealista.com/blur/WEB_LISTING/0/...,367458,21,NaN,298000.0,flat,sale,110.0,True,...,flat,"Monte Alto - Zalaeta - Atocha, A Coruña",Piso en Calle San Amaro,NaN,True,True,NaN,NaN,NaN,NaN
